<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/10.mapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/10.mapi.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

#  Model Alignment, Prompting, and In-Context Learning  
📝 SALP chapter 12

- **Prompting LLMs with Natural Language**: 
  - Large language models (LLMs) can perform tasks like translation, summarization, and sentiment analysis through `natural language prompts`, which act as `instructions` for the model.

- **In-Context Learning**: 
  - Prompts serve as a learning signal, allowing LLMs to learn new tasks through `example-based instructions` `without altering their internal parameters`.

- **Limitations of Pretraining**: 
  - LLMs are often ineffective at following instructions precisely due to insufficient pretraining; 
  - `instruction tuning`, which finetunes models on specific instructions and responses, helps improve task accuracy.

- **Safety and Harm Prevention**: 
  - Pretrained LLMs can generate harmful or unsafe outputs; 
  - addressing this involves training for safety during instruction tuning to prevent harmful language or misinformation.

- **Model Alignment**: 
  - To better align LLM behavior with human values, `preference alignment techniques` like RLHF (Reinforcement Learning from Human Feedback) are used alongside instruction tuning, 
  - enhancing model responses to align with user goals and safety standards.

## Prompting
- **Purpose of a Prompt**: A prompt is a text input that guides a language model (LLM) to perform specific tasks by providing context for generating relevant outputs.

  - Creating effective prompts for various tasks is known as `prompt engineering`, crucial for guiding LLMs to perform well.

- **Sentiment Classification Example**: 
  - For sentiment analysis, a prompt like “In short, our stay was…” can lead an LLM to generate contextually fitting completions, such as negative or positive summaries.
- 🍎 **Example**: 
  - Prompt: “Did not like the service that I was provided... In short, our stay was”
  - Output: “…not a pleasant one. The staff at the front desk were not welcoming or friendly.”

- **Task Versatility with Contextual Prompts**: LLMs can perform multiple tasks (e.g., summarization, translation, truthfulness check) by simply adjusting the prompt context.
  - With the right prompts, a single LLM can adapt to different tasks, requiring minimal additional information to produce relevant responses.

- **Prompt Templates for Efficiency**: Using templates for tasks like summarization, translation, and sentiment analysis reduces the need to create new prompts for each task.
  - Templates include an `input placeholder followed by task-specific text`, guiding LLMs to generate targeted outputs.

  | Task                  | Prompt Template                               |
  |-----------------------|-----------------------------------------------|
  | **Summarization**     | `{input} ; tldr;`                            |
  | **Translation**       | `{input} ; translate to French:`             |
  | **Sentiment**         | `{input}; Overall, it was`                   |
  | **Fine-Grained Sentiment** | `{input}; What aspects were important in this review?` |

  - Templates are `populated with actual input` text, creating `filled prompts` suitable for the LLM to process.
  - Positioning task `instructions at the end` of the prompt helps direct LLMs to produce desired outputs.
  - Unclear prompts can lead to unexpected outputs, as seen in cases where prompts don't specify clear constraints.
- **Structured Answer Prompts**: Some prompts `specify answer choices` (e.g., positive or negative sentiment), making outputs more predictable and structured.
  - Prompts can `enforce constraints` like response length, role-playing (persona responses), or structured outputs in formats like JSON.
- **Instruction Tuning**: More complex prompts, such as `chain-of-thought` reasoning, prompt
the system to `break down complex tasks`.
- **Prompt Workflow**: Tasks are processed by 
  - selecting or designing a template with a `free parameter` for the input text 
  - filling it with input, 
  - generating output via autoregressive decoding, 
  - and using or refining the result for specific needs.

- 🍎**Example**:
  - Prompt Template for Sentiment: `{input};` Overall, it was
  - Filled Prompt: “`Did not like the service…` Overall, it was”
  - Expected Output: “…not enjoyable”

### Learning from Demonstrations: Few-Shot Prompting
- `Few-shot prompting` involves adding labeled examples (demonstrations) to prompts
  - contrasting with zero-shot prompting which has no examples.
- `A small number of demonstrations` generally improves model performance, 
  - with the first example yielding the most benefit 
  - and diminishing returns from additional examples.
- Including too many demonstrations can reduce generalization, 
  - as models might overfit to specific example details instead of learning task patterns.
- Demonstrations should ideally be `similar` to the current input, 
  - dynamically retrieved based on similarity, such as using embeddings.
- Frameworks like [DSPy](https://drchrislevy.github.io/posts/dspy/dspy.html) can `programmatically optimize prompt performance` by identifying the best set of demonstrations to include based on task performance metrics.

### In-Context Learning and Induction Heads
- Prompting, even without demonstrations, can enhance a model's ability to predict upcoming tokens, a process termed `in-context learning`.
- `In-context learning` refers to a model’s capacity to improve task performance or predictive accuracy based `solely on information within a prompt context`.
- `Induction heads`, specialized circuits in transformer attention mechanisms, are hypothesized to drive in-context learning by `recognizing and predicting repeated patterns`.
- Induction heads function by identifying patterns like "AB...A" and predicting "B" to complete the sequence with a `prefix matching` component, leveraging a `copying mechanism` for prediction accuracy.
  
  ![preﬁx matching mechanism](./images/mlm/copy.png)

- [Olsson et al. (2022)](https://arxiv.org/abs/2209.11895) suggest a `generalized fuzzy matching rule`: A*B*...A→B*, 
  - where A*≈A and B*≈B (b ≈ means semantically similar)
  - allowing semantic similarity in repeated sequences, may underpin in-context learning.
- Ablation studies by [Crosbie and Shutova (2022)](https://arxiv.org/abs/2407.07011) show that removing induction heads significantly reduces in-context learning performance, suggesting their critical role in prompt-based learning.

### Post-training and Model Alignment
- Limitations exist due to LLMs’ main training goal of `predicting the next word`, which may lead to errors in following specific instructions.
  - 🍎 [Examples](https://arxiv.org/abs/2203.02155) show LLMs may misunderstand prompts, like failing to translate or explain in child-friendly terms, instead generating unrelated text continuations.

  ```python
  Prompt: Explain the moon landing to a six year old in a few sentences.
  Output: Explain the theory of gravity to a 6 year old.

  Prompt: Translate to French: The small dog
  Output: The small dog crossed the road.
  ```

- LLMs can produce harmful content, including misinformation, hate speech, and [stereotypes](https://doi.org/10.18653/v1/2023.acl-long.84), often even from [neutral prompts](https://doi.org/10.18653/v1/2020.findings-emnlp.301).
- The `primary issue is the misalignment` between the LLMs’ word-prediction objective and the human need for safe, helpful responses.
- Two additional training approaches—`instruction tuning` and `preference alignment` (such as Reinforcement Learning from Human Feedback, RLHF)—help `align LLMs with human needs`.
- `Base model` refers to an LLM before these alignment processes, which are applied `post-training` to improve safety and utility.

## Model Alignment: Instruction Tuning
- `Instruction tuning` enhances an LLM’s ability to follow diverse instructions across tasks.
  - It fine-tunes a pretrained LLM on `task-specific instruction-response` pairs, improving general instruction-following.
- Treated as `supervised learning`, it trains the model to predict the next token using labeled instructions as training data.
  - Despite using the same language modeling objective as pretraining, instruction tuning is considered supervised because each instruction has a correct answer or response.
- It differs from domain adaptation, where the LLM is fine-tuned on new domain data with all parameters updated.

  ![Instruction tuning compared to the other kinds of finetuning.](./images/mapi/inst.png)

- Parameter-efficient fine-tuning, such as LoRA, involves adding small new parameters specific to a domain while freezing the main model parameters.
- Task-based fine-tuning adapts the model to particular tasks (like classification) with specialized classification heads, which may or may not involve updating the model parameters.
- Instruction tuning continues training on instruction data, but without adding new parameters or task-specific components.
- Instruction tuning is cost-effective, requiring far fewer resources than initial LLM pretraining, with training typically limited to several epochs over instruction datasets.

### Instructions as Training Data
